In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Read Parquet from MinIO") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "password") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()


24/11/19 12:58:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
parquet_df = spark.read.parquet("s3a://warehouse/filtered_data.parquet")


24/11/19 12:58:24 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


AnalysisException: [PATH_NOT_FOUND] Path does not exist: s3a://warehouse/filtered_data.parquet.

In [4]:
parquet_df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2022-01-01 00:36:48|  2022-01-01 01:14:20|            1.0|          4.3|       1.0|                 N|          68|         163|           1|       23.5|  0.5|    0.5|       3.

In [5]:
parquet_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



# Pandas

In [6]:
import pandas as pd

# ตั้งค่า S3 Credentials
s3_options = {
    "key": "admin",
    "secret": "password",
    "client_kwargs": {
        "endpoint_url": "http://minio:9000"
    }
}


In [8]:
pip install s3fs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 23.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.2/319.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 15.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
# อ่านไฟล์ Parquet จาก MinIO
df = pd.read_parquet("s3://warehouse/filtered_data.parquet", storage_options=s3_options)

# แสดงข้อมูลตัวอย่าง
print(df.head())


   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2022-01-01 00:36:48   2022-01-01 01:14:20              1.0   
1         1  2022-01-01 00:40:15   2022-01-01 01:09:48              1.0   
2         2  2022-01-01 18:04:06   2022-01-01 18:40:52              1.0   
3         2  2022-01-01 00:55:48   2022-01-01 01:14:24              1.0   
4         1  2022-01-01 00:10:27   2022-01-01 00:41:59              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           4.30         1.0                  N            68           163   
1          10.30         1.0                  N           138           161   
2           9.70         1.0                  N           138            48   
3           6.67         1.0                  N           138           229   
4          17.10         2.0                  N           132           170   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


# PyArrow

In [16]:
import s3fs
import pyarrow as pa
import pyarrow.parquet as pq

# ตั้งค่า FileSystem ของ S3
fs = s3fs.S3FileSystem(key='admin', secret='password', client_kwargs={'endpoint_url': 'http://minio:9000'})

# ดึงรายชื่อไฟล์ในโฟลเดอร์
folder_path = "s3://warehouse/filtered_data.parquet"
files = fs.glob(f"{folder_path}/*.parquet")  # ค้นหาเฉพาะไฟล์ .parquet

print("Files found:", files)  # แสดงรายชื่อไฟล์ที่พบ

# อ่านไฟล์ทั้งหมดและรวมข้อมูล
tables = [pq.read_table(file, filesystem=fs) for file in files]
combined_table = pa.concat_tables(tables)

# แปลงเป็น Pandas DataFrame
df = combined_table.to_pandas()
print(df.head())


Files found: ['warehouse/filtered_data.parquet/part-00000-c8da6913-8c25-43c0-bf39-00139bd8e321-c000.snappy.parquet', 'warehouse/filtered_data.parquet/part-00003-c8da6913-8c25-43c0-bf39-00139bd8e321-c000.snappy.parquet']
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2022-01-01 00:36:48   2022-01-01 01:14:20              1.0   
1         1  2022-01-01 00:40:15   2022-01-01 01:09:48              1.0   
2         2  2022-01-01 18:04:06   2022-01-01 18:40:52              1.0   
3         2  2022-01-01 00:55:48   2022-01-01 01:14:24              1.0   
4         1  2022-01-01 00:10:27   2022-01-01 00:41:59              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           4.30         1.0                  N            68           163   
1          10.30         1.0                  N           138           161   
2           9.70         1.0                  N           138            48   
3           6